In [1]:
import os
import sys
import time

import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from tqdm import tqdm

In [2]:
# url is like "https://www.vogue.com/fashion-shows/designer/chanel"
def get_year_season(driver):
    # click the 'Show More' to view all of years, seasons collection
    button = "BaseButton-azpcp ButtonWrapper-dPwOur cTnZjR cdLMwc button button--primary-pair SummaryGridCompactExpandButton-cwjZmN kmRMDf"
    cls = button.replace(' ', '.')
    more = driver.find_elements(By.CLASS_NAME, cls)
    more
    while more:
        more[0].click()
        more = driver.find_elements(By.CLASS_NAME, cls)
        time.sleep(0.1)

    # class_grid
    grid = 'GridItem-bwmuQH bpJgOF grid--item'
    cls = grid.replace(' ', '.')

    # year, season collections
    items = driver.find_elements(By.CLASS_NAME, cls)

    arr_year_season = []
    for item in items:
        # Ex> 'Spring 2023 Ready-to-Wear'
        name_collection = item.text.replace(' ', '_')
        # Ex> 'https://www.vogue.com/fashion-shows/spring-2023-ready-to-wear/chanel'
        rink = item.find_element(By.CSS_SELECTOR, 'a')
        arr_year_season.append(tuple((name_collection, rink.get_attribute('href'))))
    return arr_year_season

In [3]:
# button in a year_season website
def get_image(driver, href, pre_name_img, iterator):
    # class name of button
    button = 'BaseButton-azpcp ButtonWrapper-dPwOur cTnZjR cdLMwc button button--primary-pair GalleryThumbnailExpandButton-fxLmiW kguvPC'
    cls = button.replace(' ', '.')
   
    driver.get(href)

    # click all of 'load more'
    more = driver.find_elements(By.CLASS_NAME, cls)
    while more:
        more[0].click()
        time.sleep(0.1)
        more = driver.find_elements(By.CLASS_NAME, cls)

    # class name of images grid
    grid = "GridItem-bwmuQH bpJgOF grid--item"
    cls = grid.replace(' ', '.')        
    items = driver.find_elements(By.CLASS_NAME, cls)
    for item in items:
        img = requests.get(item.find_element(By.CSS_SELECTOR, 'img').get_attribute('src'))            
        filename = '_'.join(item.text.split())
        if 'Detail_Look' in filename:
            return

        # tqdm log
        log = 'image name : ' + filename
        iterator.set_postfix_str(log) 
        
        # save the images in dir_save
        with open(pre_name_img + '_' + filename+'.png', 'wb') as f:
            f.write(img.content)

In [4]:
# chrome browser
path_driver = r'/Users/jujemu/Documents/github/capstone/chromedriver'
driver = webdriver.Chrome(service=Service(path_driver))

# url of brand
# edit below url += '{brand}'
brand = 'christian-dior'
url = 'https://www.vogue.com/fashion-shows/designer/'
url += brand

dir_save = r'./rsc/imgs/'
dir_save += brand
os.makedirs(dir_save, exist_ok=True)

# open browser using selenium webdriver
# if you want, options,headless, is useful, which skips browser window
driver.get(url)

In [5]:
arr_year_season = get_year_season(driver)

In [6]:
with tqdm(arr_year_season, desc="Crawling images", file=sys.stdout) as iterator:
    for name_ys, href in iterator:
        pre_name = os.path.join(dir_save, name_ys)
        get_image(driver, href, pre_name, iterator)

Crawling images: 100%|██████████| 127/127 [1:22:35<00:00, 39.02s/it, image name : Look_37]          


In [7]:
len(os.listdir(dir_save))

7289